In [1]:
import re
import csv
import pandas as pd 

from utils import *

2025-04-20 12:42:08.298 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:42:08.320 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
# Regular expression pattern to match simplified and traditional characters
pattern = re.compile(r'^(.+?)〔(.+?)〕$')

# Prepare the CSV rows with header
csv_rows = [['simplified', 'traditional']]
file_in = "simplified-1986.md"
file_out = "simplified-1986.csv"

uniq_ = set()
n = 0
m = 0
# Read and parse the input file
with open(file_in, 'r', encoding='utf-8') as file:
    for line in file:
        # Split line into entries using tab as separator
        entries = line.strip().split()
        for entry in entries:
            entry = entry.strip()
            # Match each entry against the pattern
            match = pattern.match(entry)
            if match:
                simplified = match.group(1).strip()
                traditional = match.group(2).strip().split("、")[0]


                # if len(traditional) > 1:
                #     m += 1
                #     print(f"[{m}] multi entry: {entry}")
                
                if simplified in uniq_: 
                    n += 1
                    print(f"[{n}] dup entry: {entry}")
                else: 
                    uniq_.add(simplified)
                    csv_rows.append([simplified, traditional])
                    continue
                    
                    

# Write the parsed data to CSV
with open(file_out, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_rows)

[1] dup entry: 只〔戠〕
[2] dup entry: 账〔賬〕
[3] dup entry: 贱〔賤〕
[4] dup entry: 溅〔濺〕
[5] dup entry: 赆〔贐〕
[6] dup entry: 赉〔賚〕
[7] dup entry: 轹〔轢〕
[8] dup entry: 辆〔輛〕
[9] dup entry: 轳〔轤〕
[10] dup entry: 轮〔輪〕
[11] dup entry: 驺〔騶〕
[12] dup entry: 骅〔驊〕
[13] dup entry: 骊〔驪〕
[14] dup entry: 骖〔驂〕
[15] dup entry: 觌〔覿〕
[16] dup entry: 赎〔贖〕
[17] dup entry: 闯〔闖〕
[18] dup entry: 闼〔闥〕
[19] dup entry: 阐〔闡〕
[20] dup entry: 鸧〔鶬〕
[21] dup entry: 鸫〔鶇〕
[22] dup entry: 鸬〔鸕〕
[23] dup entry: 鹇〔鷳〕
[24] dup entry: 鹂〔鸝〕
[25] dup entry: 鹦〔鸚〕
[26] dup entry: 闿〔闓〕
[27] dup entry: 觊〔覬〕
[28] dup entry: 验〔驗〕
[29] dup entry: 签〔簽〕
[30] dup entry: 骄〔驕〕
[31] dup entry: 轿〔轎〕
[32] dup entry: 驱〔驅〕
[33] dup entry: 鸥〔鷗〕
[34] dup entry: 闱〔闈〕
[35] dup entry: 骁〔驍〕
[36] dup entry: 须〔須〕
[37] dup entry: 颅〔顱〕
[38] dup entry: 颊〔頰〕
[39] dup entry: 颟〔顢〕
[40] dup entry: 颞〔顳〕
[41] dup entry: 鲈〔鱸〕
[42] dup entry: 鲚〔鱭〕
[43] dup entry: 鲟〔鱘〕
[44] dup entry: 鲗〔鰂〕
[45] dup entry: 鲙〔鱠〕
[46] dup entry: 鲡〔鱺〕
[47] dup entry: 鲢〔鰱〕
[48] dup entry: 鲥〔鰣〕
[

In [16]:
df = pd.read_csv(file_out) 

TypeError: read_csv() got an unexpected keyword argument 'index'

In [4]:
df.shape, df.columns

((2186, 2), Index(['simplified', 'traditional'], dtype='object'))

In [25]:
df.rename(columns={"simplified":"zi"}, inplace=True)

In [26]:
df.head(5) 

,zi,traditional,is_active
0,碍,礙,Y
1,肮,骯,Y
2,袄,襖,Y
3,坝,壩,Y
4,板,闆,Y


In [27]:
df.tail(5) 

,zi,traditional,is_active
2181,娲,媧,Y
2182,祸,禍,Y
2183,脶,腡,Y
2184,窝,窩,Y
2185,蜗,蝸,Y


In [17]:
df["is_active"] = 'Y'

In [28]:
l_sim = df["zi"].to_list() 

In [29]:
l_trad = df["traditional"].to_list()

In [30]:
len(l_sim), len(set(l_sim)), len(set(l_trad))  # (2337, 2186)

(2186, 2186, 2186)

In [31]:
table_name = "t_simp_trad"

In [32]:
with DBConn() as _conn:
    df.to_sql(table_name,_conn, index=False, if_exists="replace")

In [62]:
with DBConn() as _conn:
    sql_1 = f"""
    select max(cast(u_id as int)) as max_u_id from t_zi where is_active='Y';
    """
    
    df_1 = pd.read_sql(sql_1, _conn)

In [63]:
max_u_id = df_1["max_u_id"].to_list()[0]
max_u_id

13975

In [64]:
with DBConn() as _conn:
    sql_0 = f"""
    select * from {table_name} 
    where is_active = 'Y'  
    and zi not in (
        select zi from t_zi where is_active = 'Y'  
    )
    """
    
    df_0 = pd.read_sql(sql_0, _conn).fillna("")

In [65]:
df_0

,zi,traditional,is_active
0,号,號,Y
1,叹,嘆,Y
2,䎬,䎱,Y
3,䞍,䝼,Y
4,䙌,䙡,Y
5,㳠,澾,Y
6,䥽,鏺,Y
7,㧏,掆,Y
8,㭎,棡,Y
9,䦶,䦛,Y


In [66]:
df_0["u_id"] = df_0.index + 1 + int(max_u_id)

In [67]:
df_0["layer"] = 'HSK_z'

In [68]:
df_0.to_csv("zi_extra.csv",index=False)

In [69]:
extra = df_0.to_dict("records")

In [70]:
extra 

[{'zi': '号',
  'traditional': '號',
  'is_active': 'Y',
  'u_id': 13976,
  'layer': 'HSK_z'},
 {'zi': '叹',
  'traditional': '嘆',
  'is_active': 'Y',
  'u_id': 13977,
  'layer': 'HSK_z'},
 {'zi': '䎬',
  'traditional': '䎱',
  'is_active': 'Y',
  'u_id': 13978,
  'layer': 'HSK_z'},
 {'zi': '䞍',
  'traditional': '䝼',
  'is_active': 'Y',
  'u_id': 13979,
  'layer': 'HSK_z'},
 {'zi': '䙌',
  'traditional': '䙡',
  'is_active': 'Y',
  'u_id': 13980,
  'layer': 'HSK_z'},
 {'zi': '㳠',
  'traditional': '澾',
  'is_active': 'Y',
  'u_id': 13981,
  'layer': 'HSK_z'},
 {'zi': '䥽',
  'traditional': '鏺',
  'is_active': 'Y',
  'u_id': 13982,
  'layer': 'HSK_z'},
 {'zi': '㧏',
  'traditional': '掆',
  'is_active': 'Y',
  'u_id': 13983,
  'layer': 'HSK_z'},
 {'zi': '㭎',
  'traditional': '棡',
  'is_active': 'Y',
  'u_id': 13984,
  'layer': 'HSK_z'},
 {'zi': '䦶',
  'traditional': '䦛',
  'is_active': 'Y',
  'u_id': 13985,
  'layer': 'HSK_z'},
 {'zi': '䦷',
  'traditional': '䦟',
  'is_active': 'Y',
  'u_id': 13986

In [71]:
with DBConn() as _conn:
    for row in extra: 
        insert_sql = f"""
        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('{row.get("zi")}', '{row.get("traditional")}', '{row.get("layer")}',  '{row.get("is_active")}', '{row.get("u_id")}'); 
        """
        print(insert_sql) 
        db_run_sql(insert_sql, _conn)


        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('号', '號', 'HSK_z',  'Y', '13976'); 
        
[DEBUG] 
        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('号', '號', 'HSK_z',  'Y', '13976'); 
        

        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('叹', '嘆', 'HSK_z',  'Y', '13977'); 
        
[DEBUG] 
        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('叹', '嘆', 'HSK_z',  'Y', '13977'); 
        

        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('䎬', '䎱', 'HSK_z',  'Y', '13978'); 
        
[DEBUG] 
        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('䎬', '䎱', 'HSK_z',  'Y', '13978'); 
        

        insert into t_zi (zi, traditional, layer, is_active, u_id) 
        values ('䞍', '䝼', 'HSK_z',  'Y', '13979'); 
        
[DEBUG] 
        insert into t_zi (zi, traditional, layer, is_acti

In [74]:
sql_1 = """
select z.zi,z.traditional,st.traditional  
from t_zi z join t_simp_trad st on z.zi = st.zi 
where st.traditional != z.traditional and z.is_active='Y';
"""

with DBConn() as _conn:  
    df_1 = pd.read_sql(sql_1, _conn).fillna("")

In [75]:
df_1

,zi,traditional,traditional
0,众,眾,衆
1,赃,贓,贜
2,绣,繡,綉
3,钵,缽,鉢
4,厕,廁,厠
...,...,...,...
74,脏,髒 / 臟,臟
75,荡,盪 / 蕩,蕩
76,蒙,,矇
77,说,說,説


In [76]:
df_1.to_csv("zi_extra_review.csv",index=False)

In [79]:
with DBConn() as _conn:
    upd_sql = f"""update t_ele_zi set traditional  = (
select traditional from t_simp_trad where zi = t_ele_zi.zi limit 1
) 
where is_active='Y';
    """
    db_run_sql(upd_sql, _conn)

[DEBUG] update t_ele_zi set traditional  = (
select traditional from t_simp_trad where zi = t_ele_zi.zi limit 1
) 
where is_active='Y';
    
